In [ ]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import json

## Initial Pages

In [ ]:
init_url = "https://bikroy.com"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
init_req = urllib.request.Request(init_url, headers=headers)
init_page = urllib.request.urlopen(init_req)

init_Soup = BeautifulSoup(init_page)

project_href = [i['href'] for i in init_Soup.find_all('a', href=True) if '/en/ads/' in i['href']]
print(project_href)

In [ ]:
def insert_to_file(data):
    with open("mobiles_data.json", "a+") as f:
        f.write(json.dumps(data))


def scrape_item_url(url):
    req = urllib.request.Request(url, headers=headers)
    page = urllib.request.urlopen(req)

    soup = BeautifulSoup(page)
    items_href = [i['href'] for i in soup.find_all('a', attrs={'class' : 'card-link--3ssYv gtm-ad-item'})]
    return(items_href)

def scrape_item(url, category):
    try:
        req = urllib.request.Request(url, headers=headers)
        page = urllib.request.urlopen(req)

        soup = BeautifulSoup(page)
        desc = soup.find("div", {"class": "description--1nRbz"})

        title = soup.find('h1' , attrs={'class' : 'title--3s1R8'})
        sub_title = soup.find("span", {"class":"sub-title--37mkY"})
        feature = soup.find("p", {"class": "text--Xap4O"})
        amount = soup.find("div", {"class": "amount--3NTpl"})
        meta_p = [p.get_text() for p in soup.find_all("div", {"class": "word-break--2nyVq value--1lKHt"}) if p]

        return {
            "link": url,
            "title": title.get_text() if title else '',
            "sub_title": sub_title.text.strip() if sub_title else '',
            "amount": amount.text.strip() if amount else '',
            "condition": meta_p[0] if meta_p and len(meta_p) > 0 else '',
            "brand": meta_p[1] if meta_p and len(meta_p) > 1 else '',
            "model": meta_p[2] if meta_p and len(meta_p) > 2 else '',
            "category": category,
            "edition": meta_p[3] if meta_p and len(meta_p) > 3 else '',
            "authenticity":  meta_p[4] if meta_p and len(meta_p) > 4 else '',
            "feature": feature.text.strip() if feature else '',
            "description": desc.get_text() if desc else '',
        }
    except:
        return {}

data = []

In [ ]:
for cat in project_href[:2]:
  category = cat.split("/")[-1]
  for i in range(1,200):
      items_url = scrape_item_url(
          f"{init_url}{cat}?sort=date&order=desc&buy_now=0&urgent=0&page={i}"
          if i == 1 else f"{init_url}{project_href[0]}"
      )    
      for detail in items_url:
          scrap = scrape_item(f"{init_url}{detail}", category)
          data.append(scrap)



## Save initial file to CSV Format

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(data)
df.to_csv(r'bickroy_products.csv', index=False)

## Editing values misplaced

In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("bickroy_products.csv")

df.loc[(df.edition == "Original"), ('edition', 'authenticity')] = np.NaN,"Original"

df.tail()

,link,title,sub_title,amount,condition,brand,model,category,edition,authenticity,feature,description
10741,https://bikroy.com/en/ad/itel-used-for-sale-dh...,Itel Mobile phone (Used),"Posted on 14 Oct 8:09 am, Mohammadpur, Dhaka","Tk 2,700",Used,Itel,Other model,electronics,Mobile phone,Original,"Bluetooth, Camera, Dual SIM, Expandable Memory...",New phn kinbo tai kono prblm nai 1.5 yr use ko...
10742,https://bikroy.com/en/ad/remax-ra-otg-micro-us...,Remax RA-OTG micro USB 2.0 for smartphone (ori...,"Posted on 14 Oct 8:09 am, Mirpur, Dhaka",Tk 165,New,NaN,NaN,electronics,NaN,NaN,NaN,Micro USB to OTG.OTG Micro USB AdapterNote - C...
10743,https://bikroy.com/en/ad/walton-l13-used-for-s...,Walton L13 (Used),"Posted on 14 Oct 8:09 am, Sadar Road, Barishal",Tk 450,Used,Walton,L13,electronics,NaN,Original,"Bluetooth, Camera, Dual SIM, Expandable Memory...","*মডেল ওয়ালটন L4,,, * ফোনে কোন পবলেম নাই ফোন ১০..."
10744,https://bikroy.com/en/ad/samsung-galaxy-a71-us...,Samsung Galaxy A71 (Used),"Posted on 14 Oct 8:09 am, Mirpur, Dhaka","Tk 27,000",Used,Samsung,Galaxy A71,electronics,NaN,Original,"Bluetooth, Camera, Dual SIM, Fingerprint Senso...",128/8officialmidnight black color3 moths usedb...
10745,https://bikroy.com/en/ad/vivo-y12s-3-32-phon-e...,Vivo Y12s 3/32___ফোন একদম নতুন (Used),"Posted on 14 Oct 8:08 am, Netrokona, Mymensing...","Tk 9,500",Used,Vivo,Y12s,electronics,3/32___ফোন একদম নতুন,Original,"Bluetooth, Camera, Dual-Lens Camera, Dual SIM,...","ফোন একদম নতুন,, ফোনের কোনো সমস্যা নেই,, একদম ফ..."


In [12]:
df.to_csv("bickroy_products.csv", index=False)